In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [2]:
def get_fund_price(fund_code, retries=3, delay=5):
    url = f'https://www.tefas.gov.tr/FonAnaliz.aspx?FonKod={fund_code}'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            
            price = None
            daily_return = None
            
            # Find the <li> tag that contains the "Son Fiyat (TL)" heading
            li_tags = soup.find_all('li')
            for li in li_tags:
                if 'Son Fiyat (TL)' in li.text:
                    # Find the <span> tag inside the <li> tag
                    price_span = li.find('span')
                    if price_span:
                        price = price_span.text.strip().replace(',', '.')
                if 'Günlük Getiri (%)' in li.text:
                    # Find the <span> tag inside the <li> tag
                    daily_span = li.find('span')
                    if daily_span:
                        daily_return = daily_span.text.strip().replace(',', '.')
            
            if price is not None and daily_return is not None:
                return price, daily_return
            else:
                print(f"Price or daily return not found for fund code {fund_code}, retrying...")
                time.sleep(delay)  # Wait before retrying

        except requests.exceptions.RequestException as e:
            print(f"An error occurred on attempt {attempt + 1}: {e}")
            time.sleep(delay)  # Wait before retrying

    return None, None

def fetch_fund_price_until_success(fund_code):
    while True:
        price, daily_return = get_fund_price(fund_code)
        if price is not None and daily_return is not None:
            return price, daily_return
        print(f"Retrying to fetch the price and daily return for fund code {fund_code}...")

# Get the prices and daily returns of CPU and KPC funds
cpu_price, cpu_daily_return = fetch_fund_price_until_success('CPU')
kpc_price, kpc_daily_return = fetch_fund_price_until_success('KPC')

print(f"CPU Fiyatı: {cpu_price}, CPU Günlük Getiri: {cpu_daily_return}")
print(f"KPC Fiyatı: {kpc_price}, KPC Günlük Getiri: {kpc_daily_return}")


Price or daily return not found for fund code KPC, retrying...
CPU Fiyatı: 1.436981, CPU Günlük Getiri: %-2.3366
KPC Fiyatı: 12.591706, KPC Günlük Getiri: %-0.8943
